In [27]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
import plotly.express as px
import datetime as dt

In [2]:
df = pd.read_csv('ttc-bus-delay-data-2022.csv')

In [3]:
df.shape

(27351, 10)

In [4]:
df.head(10)

Date Route   Time       Day                Location  \
0  1-Jan-22   320  02:00  Saturday        YONGE AND DUNDAS   
1  1-Jan-22   325  02:00  Saturday  OVERLEA AND THORCLIFFE   
2  1-Jan-22   320  02:00  Saturday       YONGE AND STEELES   
3  1-Jan-22   320  02:07  Saturday       YONGE AND STEELES   
4  1-Jan-22   320  02:13  Saturday       YONGE AND STEELES   
5  1-Jan-22   363  02:16  Saturday           KING AND SHAW   
6  1-Jan-22    96  02:18  Saturday         HUMBERLINE LOOP   
7  1-Jan-22   320  02:38  Saturday       STEELES AND YONGE   
8  1-Jan-22   320  02:55  Saturday       YONGE AND STEELES   
9  1-Jan-22   300  03:18  Saturday         KENNEDY STATION   

                Incident  Min Delay  Min Gap Direction  Vehicle  
0          General Delay          0        0       NaN     8531  
1              Diversion        131      161         W     8658  
2  Operations - Operator         17       20         S        0  
3  Operations - Operator          4       11         S        0  
4  Operations - Operator          4        8         S        0  
5  Operations - Operator         30       60       NaN        0  
6               Security          0        0         N     3536  
7  Operations - Operator          4        8       NaN        0  
8  Operations - Operator          4        8       NaN        0  
9     Emergency Services          0        0         E     8094

In [5]:
for col in df.columns:
    print(col, '- unique items: ', df[col].nunique())

Date - unique items:  181
Route - unique items:  223
Time - unique items:  1440
Day - unique items:  7
Location - unique items:  5942
Incident - unique items:  15
Min Delay - unique items:  375
Min Gap - unique items:  420
Direction - unique items:  12
Vehicle - unique items:  2189


In [6]:
num_col = df.select_dtypes('number')
print(num_col.describe())

          Min Delay       Min Gap       Vehicle
count  27351.000000  27351.000000  27351.000000
mean      20.298417     32.876714   5410.169427
std       49.636736     51.293913   4616.285325
min        0.000000      0.000000      0.000000
25%        9.000000     17.000000   1394.000000
50%       11.000000     22.000000   7940.000000
75%       20.000000     36.000000   8523.000000
max      999.000000    999.000000  93561.000000


In [7]:
print(df.select_dtypes('object').describe())

            Date  Route   Time     Day         Location  \
count      27351  27187  27351   27351            27351   
unique       181    223   1440       7             5942   
top     5-Jan-22     36  16:00  Friday  KIPLING STATION   
freq         298    971     87    4522              625   

                     Incident Direction  
count                   27351     21774  
unique                     15        12  
top     Operations - Operator         N  
freq                     9554      5968  


In [8]:
df.isnull().sum()

Date            0
Route         164
Time            0
Day             0
Location        0
Incident        0
Min Delay       0
Min Gap         0
Direction    5577
Vehicle         0
dtype: int64

In [9]:
df =df.drop(['Direction'], axis = 1)

In [10]:
df

Date Route   Time       Day                Location  \
0       1-Jan-22   320  02:00  Saturday        YONGE AND DUNDAS   
1       1-Jan-22   325  02:00  Saturday  OVERLEA AND THORCLIFFE   
2       1-Jan-22   320  02:00  Saturday       YONGE AND STEELES   
3       1-Jan-22   320  02:07  Saturday       YONGE AND STEELES   
4       1-Jan-22   320  02:13  Saturday       YONGE AND STEELES   
...          ...   ...    ...       ...                     ...   
27346  30-Jun-22   123  00:45  Thursday      SHERWAY GARDENS RD   
27347  30-Jun-22   102  01:08  Thursday     WARDEN AND DANFORTH   
27348  30-Jun-22    66  01:15  Thursday                 UNKNOWN   
27349  30-Jun-22    32  01:33  Thursday        RENFORTH STATION   
27350  30-Jun-22    32  01:55  Thursday     EGLINTON AND DUPLEX   

                    Incident  Min Delay  Min Gap  Vehicle  
0              General Delay          0        0     8531  
1                  Diversion        131      161     8658  
2      Operations - Operator         17       20        0  
3      Operations - Operator          4       11        0  
4      Operations - Operator          4        8        0  
...                      ...        ...      ...      ...  
27346  Operations - Operator         27       54     8088  
27347  Operations - Operator         30       60     3416  
27348  Operations - Operator         30       30        0  
27349  Cleaning - Unsanitary         18       36     1202  
27350  Cleaning - Unsanitary          8       16     1265  

[27351 rows x 9 columns]

In [11]:
df2 = df[df['Min Delay'] != 0]

In [12]:
df2.isnull().sum()

Date          0
Route        26
Time          0
Day           0
Location      0
Incident      0
Min Delay     0
Min Gap       0
Vehicle       0
dtype: int64

In [13]:
df2 = df2[df2["Route"].notnull()]

In [14]:
df2.isnull().sum()

Date         0
Route        0
Time         0
Day          0
Location     0
Incident     0
Min Delay    0
Min Gap      0
Vehicle      0
dtype: int64

In [15]:
df2['Date'] =df2['Date'].apply(pd.to_datetime)

In [16]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [17]:
Incident = df.groupby(df['Incident'])['Min Delay'].sum()/60 # To Hour for making more sence
df_Inc = pd.DataFrame({'labels': Incident.index,
                   'values': Incident.values
                  })
df_Inc.iplot(kind='bar',x = 'labels', y = 'values', title="Delay's Incident", colors = 'palegreen',  #I love this color <3
            xTitle = 'Incident', yTitle = 'Delay in Hours') 

In [18]:
Route = df.groupby(df['Route'])['Min Delay'].sum().reset_index()
Route_20 = Route.sort_values(by = ['Min Delay'], ascending = False).head(10)

In [19]:
Route_20

Route  Min Delay
127    52      15459
96     36      12728
76     32      12408
209    97      11088
66     29      11075
206    96      10246
129    54       8769
115    47       8504
141    63       8465
39     14       8384

These are the top 10 Routes with respect to the delay time. Let's see what is the name of these routes and make them more understandable. I found the name of them in this link : https://www.ttc.ca/routes-and-schedules#/listroutes/bus

In [20]:
Route_20['Route'] = Route_20['Route'] .replace({ '52':'Lawrence West', '36':'Finch West', 
                                                '32':'Eglinton West', '97':'Younge', '29':'Dufferin',
                                                '96':'Wilson', '54':'Lawrence East', '47':'Lansdowne',
                                                '63':'Ossington', '14':'Glencairn'})

In [21]:
Route_20.iplot(kind='bar',x = 'Route', y = 'Min Delay', title="Top 10 Routes which had the most delay", 
xTitle = 'Route Name', yTitle = 'Delay in Minutes',colors = 'turquoise') #Another Lovely Color

In [22]:
df_lawrence = df.loc[df['Route'] == '54']
df_lawrence_delay = df.groupby(['Incident'])['Min Delay'].sum().reset_index().sort_values(by = ['Min Delay'], ascending = False)
df_lawrence_delay

Incident  Min Delay
3                          Diversion     157108
10             Operations - Operator     130990
9                         Mechanical     105045
5                      General Delay      40714
11  Road Blocked - NON-TTC Collision      23692
12                          Security      21881
2                    Collision - TTC      21371
13                Utilized Off Route      13231
4                 Emergency Services      12856
6                            Held By       9724
1              Cleaning - Unsanitary       8519
7                      Investigation       4808
14                            Vision       4288
0            Cleaning - Disinfection        527
8              Late Entering Service        428

In [33]:
Day = df2.groupby(df2['Day'])['Min Delay'].sum()/60 # To Hour for making more sence
df_Day = pd.DataFrame({'labels': Day.index,
                   'values': Day.values
                  })
df_Day.iplot(kind='pie',labels = 'labels', values = 'values', title="Delay based on day of the week"
             ,pull=[0.02, 0.02,0.02, 0.02,0.02, 0.02,0.02],hole = 0.3)   

In [25]:
Monday = df.loc[df['Day']== 'Monday']
Monday_grp = Monday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Tuesday = df.loc[df['Day']== 'Tuesday']
Tuesday_grp = Tuesday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Wednesday = df.loc[df['Day']== 'Wednesday']
Wednesday_grp = Wednesday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Thursday = df.loc[df['Day']== 'Thursday']
Thursday_grp = Thursday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Friday = df.loc[df['Day']== 'Friday']
Friday_grp = Friday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Saturday = df.loc[df['Day']== 'Saturday']
Saturday_grp = Saturday.groupby(['Incident'])['Min Delay'].sum().reset_index()

Sunday = df.loc[df['Day']== 'Sunday']
Sunday_grp = Sunday.groupby(['Incident'])['Min Delay'].sum().reset_index()

In [38]:
df_month = df2.groupby(df2['Date'].dt.month)['Min Delay'].sum()/60  #grouping the delay by month
df_month = df_month.reset_index()
df_month.iplot(kind = 'bar', x= 'Date', y = 'Min Delay', title = 'The total hours of delay in each month', xTitle = 'The Number of the month',
                                    yTitle = 'Hours of delay', color = 'limegreen')

In [40]:
df_Jan = df2.loc[df2['Date'].dt.month == 1]
fig = px.scatter(df_Jan, x="Date", y="Incident", size ="Min Delay")
fig.show()